In [74]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

import numpy as np

In [75]:
art_list = [1,4,4,2,3,1]
rand_list = [1,2,3,4,5]
art_list+=rand_list
art_list

[1, 4, 4, 2, 3, 1, 1, 2, 3, 4, 5]

In [79]:
# Initialize artwork list
art_list = []

# Set up options to use a custom User-Agent
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Set up the WebDriver (in this case for Chrome)
# You need to specify the path to your ChromeDriver
chrome_driver_path = r"/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/chromedriver-mac-x64/chromedriver"

# Initialize the WebDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)


# Open the website
url = f'https://www.artnet.com/auctions/all-artworks/'
driver.get(url)

time.sleep(5)

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract the product data (adjust based on SSENSE's HTML structure)
# Example of finding product descriptions
product_list = soup.find_all('div', class_='details')

art_list += product_list
    
# Close the WebDriver after scraping
driver.quit()

In [69]:
art_list

[<div class="details">
 <a href="/auctions/artists/emily-mason/winters-edge" ng-click="ensureRightPageWasSet(brick)">
 <ul>
 <li class="ng-binding"> Emily Mason</li>
 <li class="ng-binding"><em class="ng-binding">Winter's Edge</em>, 2013</li>
 <!-- ngIf: brick.IsOpening && !brick.HideEstimate -->
 <!-- ngIf: brick.IsOpening && brick.HideEstimate && !brick.isAcquire -->
 <!-- ngIf: brick.IsOnlyPurchaseNow && brick.IsInquiryNoPrice -->
 <!-- ngIf: brick.IsOpening && brick.isAcquire && brick.Estimate == '' -->
 <!-- ngIf: brick.IsOpening && brick.isAcquire && brick.Estimate != ''  && brick.HideEstimate -->
 <!-- ngIf: !brick.IsOpening && !brick.IsInquiryNoPrice --><li class="ng-binding ng-scope" ng-if="!brick.IsOpening &amp;&amp; !brick.IsInquiryNoPrice">
 46,000 USD (2 Bids)
 </li><!-- end ngIf: !brick.IsOpening && !brick.IsInquiryNoPrice -->
 <li class="ng-binding" ng-class="{'red' : brick.Remaining.Days &lt;= 0}" ng-hide="brick.IsOnlyPurchaseNow">
 3 days remaining<span class="asterisk

In [81]:
artNetDict = {}

for index, art in enumerate(art_list, 1):
    artist_element = art.find("li", class_ = "ng-binding")
    if artist_element:
        artist_name = artist_element.text
    else:
        print(f"Missing artist name")
        continue

    piece_element = art.find("em", class_ = "ng-binding")
    if piece_element:
        piece_name = piece_element.text
    else:
        print(f"Missing piece name")
        continue

    price_element = art.find('li', class_="ng-binding ng-scope").text.strip()
    if "Bids" in price_element:
        price_text, bid_text = price_element.split(" (")
        price, currency = price_text.split(" ")
        price = float(price.replace(",", ""))
        bids = int(bid_text.replace("Bids)",""))
    elif "Bid" in price_element:
        price_text, bid_text = price_element.split(" (")
        price, currency = price_text.split(" ")
        price = float(price.replace(",", ""))
        bids = 1
    else:
        price_range, currency = price_element.split(" ")
        lower_bound, upper_bound = price_range.split("—")
        lower_bound = float(lower_bound.replace(",",""))
        upper_bound = float(upper_bound.replace(",",""))
        price = np.mean([lower_bound, upper_bound])
        bids = 0
    
    print(artist_name, piece_name, price, currency, bids)
    
    artNetDict[index] = {}
    artNetDict[index]["Name"] = artist_name
    artNetDict[index]["Piece"] = piece_name
    artNetDict[index]["CurrentPrice"] = price
    artNetDict[index]["Currency"] = currency
    artNetDict[index]["NumBids"] = bids

 Emily Mason Winter's Edge 46000.0 USD 2
 Sam Gilliam Key (N.T.) 50000.0 USD 2
 Kikuo Saito Blue Train 100000.0 USD 2
 Peter Halley Cross Connect 48000.0 USD 1
 Yayoi Kusama Silver at the B... 30000.0 USD 2
 Lynne Drexler Untitled 15000.0 USD 0
 Ross Bleckner My Sister's Brain 125000.0 USD 0
 Emily Mason Neap Tide 60000.0 USD 5
 Friedel Dzubas Virgin 80000.0 USD 3
 Vivian Springford Untitled (... 90000.0 USD 0
 Janet Sobel Untitled ... 22500.0 USD 0
 Loie Hollowell Holy, Hole... 105000.0 USD 0
 Robert Motherwell Cor Anglais 125000.0 USD 0
 Jules Olitski Belshazzar Dream-2 30000.0 USD 0
 Sam Gilliam Bonnard Vacatio... 40000.0 USD 0
 Kikuo Saito Blue Whisper 90000.0 USD 1
 Sam Francis Untitled 25000.0 USD 0
 Janet Sobel Untitled ... 21000.0 USD 0
 Peter Halley Untitled (6.23.... 10000.0 USD 0
 Friedel Dzubas Hot Pursuit 12500.0 USD 0
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist name
Missing artist n

In [82]:
artNetDict

{1: {'Name': ' Emily Mason',
  'Piece': "Winter's Edge",
  'CurrentPrice': 46000.0,
  'Currency': 'USD',
  'NumBids': 2},
 2: {'Name': ' Sam Gilliam',
  'Piece': 'Key (N.T.)',
  'CurrentPrice': 50000.0,
  'Currency': 'USD',
  'NumBids': 2},
 3: {'Name': ' Kikuo Saito',
  'Piece': 'Blue Train',
  'CurrentPrice': 100000.0,
  'Currency': 'USD',
  'NumBids': 2},
 4: {'Name': ' Peter Halley',
  'Piece': 'Cross Connect',
  'CurrentPrice': 48000.0,
  'Currency': 'USD',
  'NumBids': 1},
 5: {'Name': ' Yayoi Kusama',
  'Piece': 'Silver at the B...',
  'CurrentPrice': 30000.0,
  'Currency': 'USD',
  'NumBids': 2},
 6: {'Name': ' Lynne Drexler',
  'Piece': 'Untitled',
  'CurrentPrice': 15000.0,
  'Currency': 'USD',
  'NumBids': 0},
 7: {'Name': ' Ross Bleckner',
  'Piece': "My Sister's Brain",
  'CurrentPrice': 125000.0,
  'Currency': 'USD',
  'NumBids': 0},
 8: {'Name': ' Emily Mason',
  'Piece': 'Neap Tide',
  'CurrentPrice': 60000.0,
  'Currency': 'USD',
  'NumBids': 5},
 9: {'Name': ' Friedel